In [2]:
!pip install openflexure_microscope_client

     |████████████████████████████████| 66 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 2.2 MB 10.7 MB/s eta 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.1.1
    Uninstalling Pillow-8.1.1:
      Successfully uninstalled Pillow-8.1.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import openflexure_microscope_client as ofm_client
microscope = ofm_client.find_first_microscope()

Exception: There are no microscopes advertised on the local network

In [4]:
pos = microscope.position
starting_pos = pos.copy()
pos['x'] += 100
microscope.move(pos)
assert microscope.position == pos
pos['x'] -= 100
microscope.move(pos)
assert microscope.position == starting_pos

# Check the microscope will autofocus
ret = microscope.autofocus()

# Acquire an image for sanity-checking too
image = microscope.grab_image()
f, ax = plt.subplots(1,1)
ax.imshow(np.array(image))
#print(image.metadata)
print("Active microscope extensions")
for k in microscope.extensions.keys():
    print(k)


NameError: name 'microscope' is not defined